In [2]:
import sys
import os
import torch

os.environ['CUDA_DEVICE_ORDER']='PCI_BUS_ID'
os.environ['CUDA_VISIBLE_DEVICES']='2'

import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import torch.nn.functional as F

import matplotlib as mpl
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d

from equivariant_pose_graph.utils.color_utils import color_gradient

from pytorch3d.transforms import Transform3d, Rotate, Translate

%load_ext autoreload
%autoreload 2

In [3]:
def toDisplay(x, target_dim = None):
    while(target_dim is not None and x.dim() > target_dim):
        x = x[0]
    return x.detach().cpu().numpy()

In [4]:
from pytorch3d.transforms import Transform3d,Rotate, rotation_6d_to_matrix, se3_exp_map
from equivariant_pose_graph.utils.se3 import *
from equivariant_pose_graph.utils.display import *
from equivariant_pose_graph.dataset.point_cloud_dataset import PointCloudDataset

In [8]:
from equivariant_pose_graph.models.transformer_flow import ResidualFlow_DiffEmbTransformer
from equivariant_pose_graph.models.multimodal_transformer_flow import Multimodal_ResidualFlow_DiffEmbTransformer, Multimodal_ResidualFlow_DiffEmbTransformer_WithPZCondX
from equivariant_pose_graph.training.flow_equivariance_training_module_nocentering_eval_init import EquivarianceTestingModule
from equivariant_pose_graph.training.flow_equivariance_training_module_nocentering_multimodal import EquivarianceTrainingModule, EquivarianceTrainingModule_WithPZCondX

models = [
#     {
#         "name": "1 DGCNN + 1 mug overfit + inverse kl",
#         "fn": "/home/jwang/code/equivariant_pose_graph/logs/residual_flow_occlusion/2023-01-12_152242/residual_flow_occlusion/3pgp6833/checkpoints/epoch_3_global_step_1000.ckpt",
#         "mugs": [0, 9990],
#         "encoder_type": "1_dgcnn"
#     },

#     {
#         "name": "1 DGCNN + 2 mug overfit + inverse kl",
#         "fn":"/home/jwang/code/equivariant_pose_graph/logs/residual_flow_occlusion/2023-01-12_155828/residual_flow_occlusion/2nswo2uu/checkpoints/epoch_35_global_step_9000.ckpt",
#         "mugs": [0, 9990, 2, 9992],
#         "encoder_type": "1_dgcnn"
#     },
    
#     {
#         "name": "1 DGCNN + 3 mug overfit + inverse kl",
#         "fn":"/home/jwang/code/equivariant_pose_graph/logs/residual_flow_occlusion/2023-01-12_155847/residual_flow_occlusion/1rbhvbfr/checkpoints/epoch_11_global_step_3000.ckpt",
#         "mugs": [0, 9990, 2, 9992, 3, 9993],
#         "encoder_type": "1_dgcnn"
#     },
    
#     {
#         "name": "1 DGCNN + all mugs + inverse kl",
#         "fn": "/home/jwang/code/equivariant_pose_graph/logs/residual_flow_occlusion/2023-01-12_130619/residual_flow_occlusion/1mmwz1ae/checkpoints/epoch_31_global_step_8000.ckpt",
#         "mugs": [0, 9990, 1, 9991, 2, 9992, 3, 9993, 4, 9994, 6, 9996, 8, 9998, 10, 99910, 11, 99911, 14, 99914, 22, 99922, 28, 99928],
#         "encoder_type": "1_dgcnn"
#     },
    
#     {
#         "name": "2 DGCNNs + 1 mug overfit + inverse kl",
#         "fn": "/home/jwang/code/equivariant_pose_graph/logs/residual_flow_occlusion/2023-01-17_145122/residual_flow_occlusion/23n57sp0/checkpoints/epoch_31_global_step_8000.ckpt",
#         "mugs": [0, 9990],
#         "encoder_type": "2_dgcnn"
#     },

#     {
#         "name": "2 DGCNNs + 2 mug overfit + inverse kl",
#         "fn":"/home/jwang/code/equivariant_pose_graph/logs/residual_flow_occlusion/2023-01-13_140201/residual_flow_occlusion/2ofq050u/checkpoints/epoch_31_global_step_8000.ckpt",
#         "mugs": [0, 9990, 2, 9992],
#         "encoder_type": "2_dgcnn"
#     },
    
#     {
#         "name": "2 DGCNNs + 3 mug overfit + inverse kl",
#         "fn":"/home/jwang/code/equivariant_pose_graph/logs/residual_flow_occlusion/2023-01-17_154329/residual_flow_occlusion/3cejcxfp/checkpoints/epoch_35_global_step_9000.ckpt",
#         "mugs": [0, 9990, 2, 9992, 3, 9993],
#         "encoder_type": "2_dgcnn"
#     },
    
#     {
#         "name": "2 DGCNNS + all mugs + inverse kl",
#         "fn": "/home/jwang/code/equivariant_pose_graph/logs/residual_flow_occlusion/2023-01-19_082202/residual_flow_occlusion/19jqif0v/checkpoints/epoch_11_global_step_3000.ckpt",
#         "mugs": [0, 9990, 1, 9991, 2, 9992, 3, 9993, 4, 9994, 6, 9996, 8, 9998, 10, 99910, 11, 99911, 14, 99914, 22, 99922, 28, 99928],
#         "encoder_type": "2_dgcnn"
#     },
    
    {
        'name': "2 DGCNNS + all mugs + inverse kl + more certain p(z|Y)",
        "fn": "/home/jwang/code/equivariant_pose_graph/logs/residual_flow_occlusion/2023-01-20_002146/residual_flow_occlusion/2ajhzffe/checkpoints/epoch_19_global_step_5000.ckpt",
        "mugs": [0, 9990, 1, 9991, 2, 9992, 3, 9993, 4, 9994, 6, 9996, 8, 9998, 10, 99910, 11, 99911, 14, 99914, 22, 99922, 28, 99928],
        "encoder_type": "2_dgcnn"
    }
    
]

for model_data in models:
    print("\n\n\n\n", model_data["name"])
    inner_network = ResidualFlow_DiffEmbTransformer(
                        emb_nn='dgcnn', return_flow_component=True, center_feature=True,
                        inital_sampling_ratio=1, input_dims=4)
    place_nocond_network = Multimodal_ResidualFlow_DiffEmbTransformer(
                        inner_network, gumbel_temp=0.5, center_feature=True)
    place_network = Multimodal_ResidualFlow_DiffEmbTransformer_WithPZCondX(
                        place_nocond_network, encoder_type=model_data['encoder_type'])
    place_nocond_model = EquivarianceTrainingModule(
        place_nocond_network,
        lr=1e-4,
        image_log_period=100,
        weight_normalize='softmax', #'l1',
        softmax_temperature=1
    )
    place_model = EquivarianceTrainingModule_WithPZCondX(
        place_network,
        place_nocond_model,
    )


    place_model.cuda()
    place_model.load_state_dict(torch.load(model_data["fn"])['state_dict']);
    
    #######
    
    for mug_id in model_data['mugs']:
        from torch.utils.data import DataLoader

        # dataset = PointCloudDataset('/home/bokorn/src/ndf_robot/data/renders', 
        # dataset = PointCloudDataset('/home/jwang/code/equivariant_pose_graph/data/bokorn_ndf_robot_data_duprack_bothmugrack/renders', 
        dataset = PointCloudDataset('/home/jwang/code/equivariant_pose_graph/data/train_data_duprack_bothmugrack/renders', 
        # dataset = PointCloudDataset('/home/jwang/code/equivariant_pose_graph/data/train_data_duprack_bothmugrack_overfit/renders', 
                                    cloud_type='teleport',
        #                             cloud_type='init',
                                    dataset_indices=[mug_id])
        #                             dataset_indices=[3])
        dataloader = DataLoader(dataset)
        
        from pytorch3d.transforms import Transform3d
        
        #######
        
        from pytorch_lightning.utilities.seed import seed_everything

        seed_everything(0)

        for data in dataloader:
            points_action = data['points_action'].cuda()[:,:-2]
            points_anchor = data['points_anchor'].cuda()
            points_action_trans = data['points_action_trans'].cuda()[:,:-2]
            points_anchor_trans = data['points_anchor_trans'].cuda()
            break
        
        from equivariant_pose_graph.utils.se3 import random_se3
        kls = []
        for i in range(100):
            T0 = random_se3(1, rot_var=180, trans_var=0.5).cuda()
            T1 = random_se3(1, rot_var=180, trans_var=0.5).cuda()
            points_action_trans = T0.transform_points(points_action)
            points_anchor_trans = T1.transform_points(points_anchor)
            

            #####

            vis_train = True

            if vis_train:
                place_model.train()
                res = place_model.get_transform(points_action_trans, points_anchor_trans,  points_action, points_anchor)

                pred_T_action = res['pred_T_action']
                flow_action = res['flow_components']['flow_action'][:,:,:3]
                pred_w_action = torch.sigmoid(res['flow_components']['flow_action'][:,:,3])
                flow_anchor = res['flow_components']['flow_anchor'][:,:,:3]
                pred_w_anchor = torch.sigmoid(res['flow_components']['flow_anchor'][:,:,3])
                corr_flow_action = res['flow_components']['corr_flow_action']
                corr_flow_anchor = res['flow_components']['corr_flow_anchor']
                corr_points_action = res['flow_components']['corr_points_action']
                corr_points_anchor = res['flow_components']['corr_points_anchor']
                residual_flow_action = res['flow_components']['residual_flow_action']
                residual_flow_anchor = res['flow_components']['residual_flow_anchor']
                action_attn = res['flow_components']['action_attn']
                anchor_attn = res['flow_components']['anchor_attn']
                goal_emb_action = res['flow_components']['goal_emb'][:, :, :residual_flow_action.shape[1]]
                goal_emb_anchor = res['flow_components']['goal_emb'][:, :, residual_flow_action.shape[1]:]
                dense_trans_pt_action = res['flow_components']['dense_trans_pt_action']
                dense_trans_pt_anchor = res['flow_components']['dense_trans_pt_anchor']
                trans_pt_action = res['flow_components']['trans_pt_action']
                trans_pt_anchor = res['flow_components']['trans_pt_anchor']
                trans_sample_action = res['flow_components']['trans_sample_action']
                trans_sample_anchor = res['flow_components']['trans_sample_anchor']
                goal_emb_cond_x_action = res['flow_components']['goal_emb_cond_x'][:, :, :residual_flow_action.shape[1]]
                goal_emb_cond_x_anchor = res['flow_components']['goal_emb_cond_x'][:, :, residual_flow_action.shape[1]:]
            else:
                place_model.eval()
                res = place_model.training_module_no_cond_x.get_transform(points_action_trans, points_anchor_trans, points_action, points_anchor)
                goal_emb_action = res['flow_components']['goal_emb'][:, :, :residual_flow_action.shape[1]]
                goal_emb_anchor = res['flow_components']['goal_emb'][:, :, residual_flow_action.shape[1]:]

                res = place_model.get_transform(points_action_trans, points_anchor_trans, points_action, points_anchor)
                pred_T_action = res['pred_T_action']
                flow_action = res['flow_components']['flow_action'][:,:,:3]
                pred_w_action = torch.sigmoid(res['flow_components']['flow_action'][:,:,3])
                flow_anchor = res['flow_components']['flow_anchor'][:,:,:3]
                pred_w_anchor = torch.sigmoid(res['flow_components']['flow_anchor'][:,:,3])
                corr_flow_action = res['flow_components']['corr_flow_action']
                corr_flow_anchor = res['flow_components']['corr_flow_anchor']
                corr_points_action = res['flow_components']['corr_points_action']
                corr_points_anchor = res['flow_components']['corr_points_anchor']
                residual_flow_action = res['flow_components']['residual_flow_action']
                residual_flow_anchor = res['flow_components']['residual_flow_anchor']
                action_attn = res['flow_components']['action_attn']
                anchor_attn = res['flow_components']['anchor_attn']
                dense_trans_pt_action = res['flow_components']['dense_trans_pt_action']
                dense_trans_pt_anchor = res['flow_components']['dense_trans_pt_anchor']
                trans_pt_action = res['flow_components']['trans_pt_action']
                trans_pt_anchor = res['flow_components']['trans_pt_anchor']
                trans_sample_action = res['flow_components']['trans_sample_action']
                trans_sample_anchor = res['flow_components']['trans_sample_anchor']
                goal_emb_cond_x_action = res['flow_components']['goal_emb_cond_x'][:, :, :residual_flow_action.shape[1]]
                goal_emb_cond_x_anchor = res['flow_components']['goal_emb_cond_x'][:, :, residual_flow_action.shape[1]:]

            ########

            post_action_pts = toDisplay(pred_T_action.transform_points(points_action_trans))[0]
            post_anchor_pts = toDisplay(pred_T_action.inverse().transform_points(points_anchor_trans))[0]

            action_pts = toDisplay(points_action)[0]
            anchor_pts = toDisplay(points_anchor)[0]
            action_trans_pts = toDisplay(points_action_trans)[0]
            anchor_trans_pts = toDisplay(points_anchor_trans)[0]
            action_flow = toDisplay(flow_action)[0]
            action_flow_corr = toDisplay(corr_flow_action)[0]
            action_flow_res = toDisplay(residual_flow_action)[0]
            anchor_flow = toDisplay(flow_anchor)[0]
            # vis_scores = toDisplay(scores)[0]
            vis_attn_act = toDisplay(action_attn)[0]
            vis_attn_anc = toDisplay(anchor_attn)[0]
            action_w = toDisplay(pred_w_action)[0]
            anchor_w = toDisplay(pred_w_anchor)[0]

            #######


            only_mug = False
            if only_mug:
                def test(goal_emb_cond_x, goal_emb):
                    B, K, D = goal_emb.shape
                #     print(goal_emb_cond_x[:, :, :D//2].shape)
                #     print(goal_emb[:, :, :D//2].shape)
                    goal_emb_loss = F.kl_div(F.log_softmax(goal_emb_cond_x[:, :, :], dim=-1), F.log_softmax(goal_emb[:, :, :], dim=-1), log_target=True, reduction='batchmean')
                #                 + F.kl_div(F.log_softmax(goal_emb_cond_x[:, :, D//2:], dim=-2), F.log_softmax(goal_emb[:, :, D//2:], dim=-2), log_target=True, reduction='batchmean')
                #     return F.kl_div(F.log_softmax(goal_emb_cond_x.reshape(B, K, D, 1), dim=-2), F.log_softmax(goal_emb.reshape(B, K, D, 1), dim=-2), log_target=True, reduction='batchmean')
                    return goal_emb_loss

                kls.append(test(goal_emb_cond_x_action, goal_emb_action).item())
            else:
                def test(goal_emb_cond_x, goal_emb):
                    B, K, D = goal_emb.shape
                #     print(goal_emb_cond_x[:, :, :D//2].shape)
                #     print(goal_emb[:, :, :D//2].shape)
                    goal_emb_loss = F.kl_div(F.log_softmax(goal_emb_cond_x[:, :, :], dim=-1), F.log_softmax(goal_emb[:, :, :], dim=-1), log_target=True, reduction='batchmean') \
                #                 + F.kl_div(F.log_softmax(goal_emb_cond_x[:, :, D//2:], dim=-2), F.log_softmax(goal_emb[:, :, D//2:], dim=-2), log_target=True, reduction='batchmean')
                #     return F.kl_div(F.log_softmax(goal_emb_cond_x.reshape(B, K, D, 1), dim=-2), F.log_softmax(goal_emb.reshape(B, K, D, 1), dim=-2), log_target=True, reduction='batchmean')
                    return goal_emb_loss

#                 if i == 0:
#                     temp = points_anchor.detach()
#                     temp, _ = temp.sort(dim=1)
#                     print(temp)
                kls.append((test(goal_emb_cond_x_anchor, goal_emb_anchor).item(), test(goal_emb_cond_x_action, goal_emb_action).item()))

        if only_mug:
            print(f"Mug KL for Mug #{mug_id}: {sum(kls) / len(kls):.3f}")
        else:
            print(f"Rack KL: {sum([a[0] for a in kls]) / len(kls):.3f}, Mug KL for Mug #{mug_id}: {sum([a[1] for a in kls]) / len(kls):.3f}")






 2 DGCNNS + all mugs + inverse kl + more certain p(z|Y)


Global seed set to 0


/home/jwang/code/equivariant_pose_graph/data/train_data_duprack_bothmugrack/renders/0_teleport_obj_points.npz


Global seed set to 0


Rack KL: 4.685, Mug KL for Mug #0: 1.278
/home/jwang/code/equivariant_pose_graph/data/train_data_duprack_bothmugrack/renders/9990_teleport_obj_points.npz


...

Rack KL: 5.070, Mug KL for Mug #22: 1.262
/home/jwang/code/equivariant_pose_graph/data/train_data_duprack_bothmugrack/renders/99922_teleport_obj_points.npz


Global seed set to 0


Rack KL: 4.042, Mug KL for Mug #99922: 0.677
/home/jwang/code/equivariant_pose_graph/data/train_data_duprack_bothmugrack/renders/28_teleport_obj_points.npz


Global seed set to 0


Rack KL: 4.997, Mug KL for Mug #28: 0.982
/home/jwang/code/equivariant_pose_graph/data/train_data_duprack_bothmugrack/renders/99928_teleport_obj_points.npz
Rack KL: 4.429, Mug KL for Mug #99928: 0.743
